In [35]:
import requests 
import pandas as pd 
import numpy as np 
import random 

from bs4 import BeautifulSoup
!pip install geopy
from geopy.geocoders import Nominatim 

from IPython.display import Image 
from IPython.core.display import HTML 
    

from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### 1- Built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name

In [44]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050"
data_toronto = requests.get(url).text
soup = BeautifulSoup(data_toronto, 'xml')

In [45]:
table=soup.find('table')

In [46]:
column_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)


In [48]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data


df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [50]:
df=df[df['Borough']!='Not assigned']

In [52]:
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [53]:
df_2=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
df_2=df_2.reset_index(drop=False)
df_2.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [88]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

df_merge.drop_duplicates(inplace=True)

df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)
df_merge = df_merge.rename(columns = {'Postalcode':'PostalCode'}) 

df_merge.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,"Parkwoods, Parkwoods"
2,M4A,North York,"Victoria Village, Victoria Village"
4,M5A,Downtown Toronto,"Harbourfront, Harbourfront"
6,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei..."
10,M7A,Downtown Toronto,"Queen's Park, Queen's Park"


In [55]:

df_merge.shape

(103, 3)

### 2- Get the latitude and the longitude coordinates of each neighborhood

In [60]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [67]:
url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geo=pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [89]:
df_geo = df_geo.rename(columns = {'Postal Code':'PostalCode'}) 
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [92]:
df_geo_merge = pd.merge(df_merge, df_geo, on = 'PostalCode')
df_geo_merge.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,"Parkwoods, Parkwoods",43.753259,-79.329656
1,M4A,North York,"Victoria Village, Victoria Village",43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei...",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Queen's Park",43.662301,-79.389494


In [93]:
df_geo_merge.shape

(103, 5)

In [94]:
df_geo_merge['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [99]:
df_geo_merge['Label']=df_geo_merge['Borough'].replace(to_replace=['Downtown Toronto','Central Toronto','West Toronto','East Toronto','Scarborough','Etobicoke','York','Mississauga','North York','East York'],value=[1,2,3,4,5,6,7,8,9,10],inplace=False)
df_geo_merge.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Label
0,M3A,North York,"Parkwoods, Parkwoods",43.753259,-79.329656,9
1,M4A,North York,"Victoria Village, Victoria Village",43.725882,-79.315572,9
2,M5A,Downtown Toronto,"Harbourfront, Harbourfront",43.654260,-79.360636,1
3,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei...",43.718518,-79.464763,9
4,M7A,Downtown Toronto,"Queen's Park, Queen's Park",43.662301,-79.389494,1


In [100]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### 3- Clustering and the plotting of the neighbourhoods of Toronto in their Borough

In [105]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [108]:
kclusters=len(df_geo_merge.Label.unique())


map_toronto_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, cluster in zip(df_geo_merge['Latitude'], df_geo_merge['Longitude'], df_geo_merge['Label']):
    label = folium.Popup(str(df_geo_merge['Borough']) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto_clusters)

In [109]:
map_toronto_clusters